## Doughnut seminar examples

If you would like to run this notebook through collab, then please use the following link:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mdblackledge/stan_examples/blob/main/notebooks/doughnut_seminar_2023.ipynb)

### Library installation
First we need to check that all relevant libraries are installed

In [1]:
!pip install pystan arviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


### Library Import

Import all relevant libraries.  Note that for Stan to work in Jupyter, we also need nest_asyncio!

In [2]:
import nest_asyncio
nest_asyncio.apply()
import httpstan
import stan

import numpy as np
import matplotlib.pyplot as plt

## Example 1. Basic usage
In this example we will create a very simple model to demonstrate how to estimat the mean and standard deviation of data sampled from a normal distribution:
$$
x_{i} \sim \mathcal{N}(\mu, \sigma)
$$

In this initial example we will assume uniform priors for the mean and standard deviation, though note that these are "improper"!

In [4]:
# Define the model string

simple_model = """
data {
    int<lower = 1> N;  // All lines must end with a semicolon.
    array[N] real x;
}
parameters {
    real mu;
    real<lower = 0> sigma;
}
model {
    for (n in 1:N) {
        target += normal_lpdf(x[n] | mu, sigma);
    }
}
"""

# Synthesize some data
N = 40
x = np.random.normal(loc = 2.0, scale = 3.0, size = N)

# Build the model.  This will take some time the first time it is run!
model = stan.build(simple_model, data = {"N": N, "x": x})

# Draw some samples
samples = model.sample(num_samples = 5000,  # How many samples to we want?
                       num_warmup = 500,  # How many samples to discard from the beginning?
                       num_chains = 2,  # How many indpendent sample chains do we want? At least 2 for diagnostics!
                       num_thin = 1)  # Do we want to reduce correlations?

Building...



Building: 32.6s, done.Messages from stanc:
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:  50% (5500/11000)
Sampling: 100% (11000/11000)
Sampling: 100% (11000/11000), done.
Messages received during sampling:
  Gradient evaluation took 3.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.36 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.26 seconds.
  Adjust your expectations accordingly!


## A small